# Introduction to LangChain and RAG

Welcome to the first part of our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with Retrieval Augmented Generation (RAG).

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [1]:
import os

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [2]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [3]:
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [4]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [5]:
request = "I want to invest in the tech sector. What are the best options?"

response = base_model.invoke(
    [
        SystemMessage(BASE_PROMPT),
        HumanMessage(request),
    ]
)

In [6]:
Markdown(response.content)

Investing in the tech sector can be a great opportunity, given its growth potential. Here are some options to consider:

1. **Tech ETFs**: Exchange-Traded Funds like the Invesco QQQ Trust (QQQ) or the Technology Select Sector SPDR Fund (XLK) provide diversified exposure to a range of tech companies.

2. **Large Cap Tech Stocks**: Consider investing in established companies like Apple (AAPL), Microsoft (MSFT), Amazon (AMZN), and Alphabet (GOOGL). These companies have strong fundamentals and a history of growth.

3. **Emerging Tech Companies**: Look into smaller, high-growth companies in areas like cloud computing, artificial intelligence, and cybersecurity. Examples include companies like Snowflake (SNOW) or CrowdStrike (CRWD).

4. **Venture Capital or Private Equity**: If you have a higher risk tolerance, consider investing in venture capital funds that focus on early-stage tech startups.

5. **Sector-Specific Funds**: Look for mutual funds or ETFs that focus on specific tech sub-sectors, such as semiconductors (e.g., VanEck Vectors Semiconductor ETF - SMH) or fintech.

6. **REITs in Tech**: Real Estate Investment Trusts that focus on data centers or cell towers, like Digital Realty (DLR) or American Tower (AMT), can provide exposure to the tech infrastructure.

Before investing, assess your risk tolerance, investment horizon, and conduct thorough research or consult with a financial advisor.

## Understanding Retrieval Augmented Generation (RAG)

Now comes the exciting part! RAG is a technique that enhances LLM responses by giving them access to external knowledge. Instead of relying solely on the model's training data, we can provide relevant information from our own database.

### Vector Database Setup

To implement RAG, we need:
1. A collection of documents (in our case, a currated set of 1'000 articles from Bloomberg financial news)
2. A way to search these documents efficiently (vector database)
3. A method to retrieve relevant information based on queries

Here we use:
- `Chroma`: A vector database for storing and retrieving documents
- `OpenAIEmbeddings`: Converts text into vector representations

Let's first set up the global configuration for our retriever.

In [7]:
EMBEDDING_MODEL = "text-embedding-3-small"
RETRIEVAL_K = 3

We'll then define helper methods to load our documents and store them in a vector store.

In [8]:
import pickle

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def load_documents(pickle_filepath: str) -> list[Document]:
    """Load documents from a pickle file."""
    with open(pickle_filepath, "rb") as file:
        return pickle.load(file)


def initialize_vector_store(document_chunks: list[Document]) -> Chroma:
    """Reset the Chroma collection and initialize a vector store using document chunks."""
    Chroma().reset_collection()
    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    return Chroma.from_documents(documents=document_chunks, embedding=embedding_model)

Let's load our documents and inspect the first one.

In [9]:
data_dir = "../data/"
data_file = "bloomberg_financial_news_1k.pkl"

documents = load_documents(os.path.join(data_dir, data_file))

In [10]:
doc_str = f"{documents[0].metadata['Headline']}\n\n{documents[0].page_content}"
Markdown(doc_str)

Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion

Kuwait Investment Authority , the country’s sovereign wealth fund, doubled investments in the U.K. over the last 10 years to more than $24 billion. Investment in Britain was “across all asset classes, sectors and industries,” Managing Director Bader Al-Saad said in a speech in London on June 28. Kuwait Investment Office, the KIA’s London-based investment arm, now manages more than $120 billion globally compared with $27 billion 10 years ago. “Alongside the real estate subsidiary, which manages our real estate assets, we have set up an infrastructure fund which we are operating from London,” Al-Saad said. The investment office “will be looking to manage more funds, in more asset classes, and in more regions of the world in the near future,” he said. The KIA, which has about $342 billion of assets according to the Sovereign Wealth Fund Institute in California , holds stakes in Daimler AG and BP Plc. (BP/) The fund manages Kuwait’s General Reserve Fund and Future Generations Fund, and invests in local, Arab and international markets.

### Initializing the Vector Store and the Retriever

The vector store and retriever are key components of our RAG system. Here's what happens in this section:

- Initialize a new Chroma vector store with these documents
- Create a retriever that will fetch the `RETRIEVAL_K` most relevant documents according to their embedding

In [11]:
vector_store = initialize_vector_store(documents)
retriever = vector_store.as_retriever(search_kwargs={"k": RETRIEVAL_K})

The retriever acts like a smart search engine - when given a question or topic, it returns the most relevant documents from our database. It does so by finding the documents similar embeddings to the query. In LangChain, this is also done with `.invoke()`. Let's try an example.

In [12]:
retriever.invoke("tech sector market trends")

[Document(id='e39b5e49-ee59-49ef-930c-4c85b63f0440', metadata={'Headline': 'ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty', 'Link': 'http://www.bloomberg.com/news/2013-11-20/arm-sees-shift-toward-cheaper-smartphones-boosting-chip-royalty.html'}, page_content='ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they mi

### Creating the RAG System

We can now augment our basic chatbot by providing it access to the retriever using **LangChain** tools, which allow the model to:
- Query the document database if needed
- Provide an answer based on the retrieved documents

#### Creating a tool with LangChain

We can create a tool using the `@tool` decorator from **LangChain** and provide it to the model using `.bind_tools()`. The model will receive all the relevant information about the tool thanks to the decorator. This way it knows how it works and can decide when to use it.

In [13]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool


@tool
def retrieval(retrieval_query: str) -> list[Document]:
    """Retrieve documents based on a query."""
    return retriever.invoke(retrieval_query)


tools = [retrieval]
tools_by_name = {tool.name: tool for tool in tools}

In [14]:
RAG_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Query the database to help the client with their request. Give a succinct and clear response based on the information you find.
"""

rag_model = base_model.bind_tools(tools)

In [15]:
request = "I want to invest in the tech sector. What are the best options?"

rag_response = rag_model.invoke(
    [
        SystemMessage(RAG_PROMPT),
        HumanMessage(request),
    ]
)

Let's check the answer. As we can see its content is empty, but a tool call has been made.

In [16]:
print(f"Content: {rag_response.content}\n\nTool Calls: {rag_response.tool_calls}")

Content: 

Tool Calls: [{'name': 'retrieval', 'args': {'retrieval_query': 'best investment options in tech sector 2023'}, 'id': 'call_D7duSPZ82kq7ZLsMczgI1xvw', 'type': 'tool_call'}]


Let's use the retrieval tool to retrieve documents following the model's query.

In [17]:
if rag_response.tool_calls:
    tool_call = rag_response.tool_calls[0]
    tool = tools_by_name[tool_call["name"]]
    documents = tool.invoke(tool_call["args"])

    documents_str = "\n\n".join(
        [f"{doc.metadata['Headline']}\n\n{doc.page_content}\n" for doc in documents]
    )

In [18]:
Markdown(documents_str)

Mairs & Power’s Frels, Henneman Named Manager of the Year

William Frels and Mark Henneman, who run the $2.5 billion Mairs & Power Growth Fund (MPGFX) , were named domestic stock-fund managers of the year by Morningstar Inc. (MORN)  The top honor for international stock funds went to the $1.3 billion Virtus Foreign Opportunities Fund and the $6.8 billion Virtus Emerging Markets Opportunities Fund, both run by Rajiv Jain, Chicago-based Morningstar said in a statement today. The fixed-income winner was Mark Kiesel , manager of the $10.2 billion Pimco Investment Grade Corporate Bond Fund. The six-member team that runs the $1.8 billion TFS Market Neutral Fund (TFSMX) won in the alternatives category. David Giroux of the $13.7 billion T. Rowe Price Capital Appreciation Fund was named allocation fund manager of the year. Both are new categories for the research firm’s awards. The Mairs & Power Growth Fund returned an average of 5.6 percent annually over the past five years, better than 89 percent of peers, according to data compiled by Bloomberg. Last year it returned 22 percent, topping 95 percent of rivals. Of the fund’s largest 25 holdings , 18 were bought in the 1990s and 17 have been in the portfolio for at least 10 years, Morningstar said in the statement. Virtus Emerging Markets Opportunities Fund (HIEMX) gained 3.8 percent annually for the past five years, better than 94 percent of competitors. The fund returned 20 percent in 2012, besting 58 percent of peers, according to the data. Kiesel’s Pimco fund outperformed 91 percent of similar funds over the past five years, returning 10 percent annually. The fund, which topped 77 percent of rivals in 2012, had 17 percent of its money in bank debt as of Sept. 30. Market Neutral TFS Market Neutral Fund gained 4.5 percent annually for the past five years, better than 97 percent of peers. The team’s 7.8 return last year topped 84 percent of rivals, according to data compiled by Bloomberg. Morningstar defines alternative investments as those that don’t fit neatly into traditional stock or fixed-income categories. T. Rowe Price Capital Appreciation gained 6 percent a year for the past five years, ahead of 95 percent of rivals. Its largest holding was Waltham, Massachusetts-based Thermo Fisher Scientific Inc. (TMO) as of Sept. 30, the data show. Giroux’s 15 percent return in 2012 beat 82 percent of the competition.


Digital China Unit Plans $492 Million Merger With Shenzhen Techo

Digital China Holdings Ltd. (861) , a distributor of technology products, said its information services unit will merge with Shenzhen Techo Telecom Co. (000555) in a 3.02 billion yuan ($492 million) all-stock deal. Shenzhen Techo will absorb Digital China Information Service Co. in exchange for 319.6 million new shares, about a 74 percent stake of its expanded share capital, Digital China said yesterday in a Hong Kong stock exchange filing. The merger gives Digital China a separate listing for its systems integration, applications software development and professional IT services business, according to the filing. Its distribution operations have been affected by a drop in consumer demand, Miles Xie, an analyst at Bocom International Holdings Ltd., wrote in a June report. The deal is deemed a spinoff under Stock Exchange of Hong Kong rules and will require approval from the exchange, Shenzhen Techo shareholders and Chinese government regulatory agencies, Digital China said. After the merger, Shenzhen Techo plans to discontinue its existing operations, the trading of liquid crystal film display panel products, according to the filing. Digital China said it will continue to distribute IT products and provide supply chain services. Digital China, which now owns 61 percent of the services unit, will hold a 45.2 percent in Shenzhen Techo, according to the filing. Tianjin Xinrui Investment Partnership LP, which owns 18.6 percent of the unit, will get a 13.8 percent stake in Shenzhen Techo, the company said. Unit investors China Singapore Suzhou Industrial Park Ventures Co., Infinity I-China Investments Israel LP and Nanjing Huiqing Tianxia Technology Co. will get stakes of 12.3 percent, 2.2 percent and 0.7 percent respectively, according to the filing. Digital China gained 1.4 percent to HK$8.64 at the close of Hong Kong trading yesterday, before the announcement. The stock has slumped 35 percent this year, compared with the 2.5 percent decline in the city’s benchmark Hang Seng Index. Shenzhen Techo has been suspended from trading in Shenzhen since May 6.


Northern Trust Profit Rises 1.7% as Market Boosts Assets

Northern Trust Corp. (NTRS) , the third- biggest independent U.S. custody bank, said first-quarter profit rose 1.7 percent as stock-market gains boosted the value of assets the company oversees. Net income increased to $164 million, or 67 cents per diluted share, from $161.2 million, or 66 cents a share, a year earlier, the Chicago-based company said today in a statement. Results missed the 72-cent average estimate of 16 analysts surveyed by Bloomberg. “We have a better macro backdrop with higher market levels this quarter, and we saw good organic growth across all three” of the largest publicly listed custody banks, Brian Bedell, a New York-based analyst at ISI Group Inc., said in an interview before results were released. The custody banks, under pressure to improve profitability hurt by record-low interest rates, are benefiting as global equity markets climb for the second straight year. Northern Trust, led by Chairman and Chief Executive Officer Frederick H. Waddell, followed larger rivals Bank of New York Mellon Corp. (BK) and State Street Corp. (STT) with job cuts in 2012 that, combined with technology initiatives, are designed to increase annual pretax income by $250 million by the end of this year. The Standard & Poor’s 500 Index (SPX) of U.S. stocks returned 14 percent in the 12 months ended March 31, and global stocks, as measured by the MSCI ACWI Index, advanced 11 percent, with reinvested dividends. Rate Impact Low rates hurt custody banks by reducing the return they make on lending and on their own investments. The U.S. Federal Reserve has held its benchmark interest rate at zero to 0.25 percent since December 2008 in an attempt to stimulate borrowing and economic growth. Custody banks keep records, track performance and lend securities for institutional investors, including mutual funds, pension funds and hedge funds. Northern Trust also manages investments for individuals and institutions. Results were announced before the start of regular U.S. trading. Northern Trust gained 7.3 percent this year through yesterday, compared with a 15 percent advance by the Standard & Poor’s 20-company index of asset managers and custody banks. BNY Mellon is scheduled to released earnings tomorrow, followed by State Street on April 19. (Northern Trust is scheduled to hold a conference call for investors at 10 a.m. New York time. The call can be accessed at http://www.northerntrust/financialreleases .)


We can now add the tool's output to the message chain with `ToolMessage`, so the model can answer based on the retrieved documents.

*Note: Here we use the base model instead of the RAG model to limit our agent to one retrieval call. A fully autonomous agent could decide to make subsequent calls to best answer the request*

In [19]:
response = base_model.invoke(
    [
        SystemMessage(RAG_PROMPT),
        HumanMessage(request),
        rag_response,
        ToolMessage(content=documents_str, tool_call_id=tool_call["id"]),
    ]
)

In [20]:
Markdown(response.content)

For investing in the tech sector, consider the following options based on recent performance and recognition:

1. **Mairs & Power Growth Fund (MPGFX)**: This fund has been recognized as one of the top domestic stock-fund managers by Morningstar. It has returned an average of 5.6% annually over the past five years, outperforming 89% of its peers. Last year, it achieved a 22% return.

2. **Virtus Emerging Markets Opportunities Fund (HIEMX)**: This fund has also shown strong performance, gaining 3.8% annually over the past five years and returning 20% in 2012, outperforming 58% of its competitors.

3. **T. Rowe Price Capital Appreciation Fund**: This fund has gained 6% annually over the past five years, ahead of 95% of rivals, with a notable holding in Thermo Fisher Scientific Inc.

These funds are well-regarded in the tech sector and have demonstrated consistent performance, making them strong candidates for investment. Always consider your risk tolerance and investment goals before proceeding.

## Practical Tips

- Watch the temperature setting: Lower values (like 0) are usually better for factual responses
- Pay attention to the number of retrieved documents (`RETRIEVAL_K`): More isn't always better
- The system prompt is crucial: It sets the context and behavior of your bot

## Conclusion

You just learned how to create a chatbot and augment it with a retrieval tool using **LangChain**, this concludes the first part of our workshop!

In the next section, we'll discover **LangGraph** and show how it allows to build sophisticated and flexible LLMs workflows.